<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/05_CHIRPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract monthly rainfall data

## Import gee and authenticate

In [1]:
import ee

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=6bz_TXmj-TpvEseRvPHRpxiopcB3YFHfkypOWHEs8ds&tc=XZFvsrTN6EahsLr47GKvYr1GIXU0aGQzDW_ct7j09IU&cc=dbKtNdHQyhBQC_wrUYfJhxjBzqbdfh7QAUK7fYI4gVg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VN3Tq6_OB3amud11b1iD9pM6huGF2MqVEbmA0FddGieiO5BfgMRw-Y

Successfully saved authorization token.


## Input arguments for data extraction

In [4]:
# Area of interest
# aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_NS_buff_5km")
# aoi_name = "MKR_NS_buff_5km"

aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_PACE")
aoi_name = "MKR_PACE"

# GEE layer ID
layer_name = "RS_012"

# Date parameters
start_year = 2012
end_year = 2022

# Range doesn't include the stop value
year_list = ee.List(list(range(start_year, end_year+1)))

## CHIRPS Daily: Climate Hazards Group InfraRed Precipitation With Station Data (Version 2.0 Final)

Spatial resolution is 5km so not sure if Matt is going to find this uesful - check first.


In [5]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")

## Rainfall processing - annual sum

In [6]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
years = ee.List.sequence(start_year, end_year)

def yearly_rain(focal_year):
  start = ee.Date.fromYMD(focal_year, 1, 1)
  end = ee.Date.fromYMD(focal_year, 12, 31)
  date_range = ee.DateRange(start, end)
  return chirps \
        .filterDate(date_range) \
        .sum() \
        .clip(aoi)

rain_years = ee.ImageCollection.fromImages(years.map(yearly_rain).flatten())
print(rain_years.getInfo())
rain_bands = rain_years.toBands()
print(rain_bands.getInfo())
print(rain_bands.bandNames().getInfo())

names_from = rain_bands.bandNames().getInfo()
names_to = ["{year}".format(year=i) for i in range(start_year, end_year + 1)]

rain_bands = rain_bands.select(names_from).rename(names_to)
print(rain_bands.bandNames().getInfo())
print(rain_bands.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'precipitation', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '0'}}, {'type': 'Image', 'bands': [{'id': 'precipitation', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '1'}}, {'type': 'Image', 'bands': [{'id': 'precipitation', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '2'}}, {'type': 'Image', 'bands': [{'id': 'precipitation', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:index': '3'}}, {'type': 'Image', 'bands': [{'id': 'precipitation', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'cr

## Export data - annual sum

In [7]:
# Get the list of band names
band_names = rain_bands.bandNames().getInfo()

# Loop through the bands and export each one as a separate image
for i in band_names:
    # Select the band
    band = rain_bands.select(i)
    output_name = f"{layer_name}_sum_{aoi_name}_{i}"
    # Define the export parameters
    export_params = {
        "image": band,
        "description": "EXPORT IMAGE TO DRIVE",
        "folder": "GEE_exports",
        "fileNamePrefix": output_name,
        "scale": 500,
        "maxPixels": 1e13,
        "fileFormat": "GeoTIFF",
        "region": aoi.geometry().bounds()
    }

    # Export the image to Google Drive
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
    print("STARTED TASK ", i)

STARTED TASK  2012
STARTED TASK  2013
STARTED TASK  2014
STARTED TASK  2015
STARTED TASK  2016
STARTED TASK  2017
STARTED TASK  2018
STARTED TASK  2019
STARTED TASK  2020
STARTED TASK  2021
STARTED TASK  2022


## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [8]:
tasks = ee.batch.Task.list()
for task in tasks[0:ee.List.length(year_list).getInfo()]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

C3256DBCPMA7FAQ2W7BXKFOF READY
GRIMYQ3T245Q3LWPYEXBYXFL READY
DTLN3U3EDSWXUNGADHTI6ZA3 READY
7AIKD2EYEJ4TQVMLNOVWGOPC READY
5C7NJJV77S44AWTDPLI52L2I READY
IWHNYTWAZMRZIHVSYKBPNZAV READY
S47Z62666JDJ3SNYJ55VKLB3 READY
RG7JYDZ3BIZECXKW2FOPJUA4 RUNNING
J6K56F4JD2VY5MBEFUSYF6QN RUNNING
4AA7KQ3JTGUPPQDSIT2ZOD55 RUNNING
2J5LPVXPTMSVIHNJ4IN6X7KE RUNNING
